# Part I

## Olympic Games

The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning.

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


#### 1. The country with the most gold medals in summer games

In [13]:
gold = df.loc[:, ['Gold']]
df[df['Gold'] == gold.max()[0]].iloc[0].name

'United States'

#### 2. The country with the biggest difference between their summer and winter gold medal counts

In [26]:
new_df = df.loc[:, ['Gold', 'Gold.1']]
new_df['# Difference'] = new_df.apply(lambda row:row['Gold'] - row['Gold.1'], axis=1)
new_df[new_df['# Difference']==new_df.max()[2]].index[0]

'United States'

#### 3. The country with the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$


In [35]:
sw = df.loc[:, ['Gold' ,'Gold.1','Gold.2']]
sw_filtered = sw[(sw['Gold']>=1) & (sw['Gold.1']>=1)]
sw_filtered['Total Gold'] = sw_filtered.apply(lambda row:row['Gold']+row['Gold.1']+row['Gold.2'], axis=1)
sw_filtered['Difference'] = sw_filtered.apply(lambda row:(row['Gold']-row['Gold.1'])/row['Total Gold'], axis=1)
sw_filtered.max()
sw_filtered[sw_filtered['Difference'] >= 0.480769].index[0]

'Bulgaria'

#### 4.  The "Points" for all of countries which is a weighted value where each gold medal (Gold.2) counts for 3 points, silver medals (Silver.2) for 2 points, and bronze medals (Bronze.2) for 1 point.

In [40]:
p = df.loc[:, ['Gold.2','Silver.2','Bronze.2']]
p['Points'] = p.apply(lambda row:(row['Gold.2']*3) + (row['Silver.2']*2) + (row['Bronze.2']*1), axis=1)
p['Points'].sort_values(ascending=False)

United States             5684
Soviet Union              2526
Great Britain             1574
Germany                   1546
France                    1500
Italy                     1333
Sweden                    1217
China                     1120
East Germany              1068
Russia                    1042
Norway                     985
Hungary                    962
Australia                  923
Finland                    895
Japan                      866
Canada                     846
Netherlands                727
Switzerland                630
South Korea                609
Romania                    572
Austria                    569
Poland                     520
West Germany               459
Cuba                       420
Bulgaria                   411
Denmark                    335
Czechoslovakia             327
Unified Team               287
Belgium                    276
United Team of Germany     269
                          ... 
Grenada                      3
Haiti   

# Part II

## Census

For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names

In [114]:
census_df = pd.read_csv('census.csv')
census_df = census_df[census_df['SUMLEV'] == 50]
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


#### 1. The state has the most counties in it

In [90]:
census_df = census_df.set_index(['STNAME'])
census_df.index.value_counts().index[0]

'Texas'

#### 2. The three most populous counties for each state, looking at the three most populous states (in order of highest population to lowest population)

In [98]:
census_df = census_df.loc[:,['CTYNAME', 'CENSUS2010POP']]
census_df = census_df.sort_values(by=['CENSUS2010POP'], ascending=[False])
census_df = census_df.reset_index()
census_df = census_df.groupby('STNAME').head(3)
census_df = census_df.set_index(['STNAME']).sum(level=0)
census_df.sort_values(by='CENSUS2010POP', ascending=False)
[census_df.sort_values(by='CENSUS2010POP', ascending=False).index[0], census_df.sort_values(by='CENSUS2010POP', ascending=False).index[1], census_df.sort_values(by='CENSUS2010POP', ascending=False).index[2]]

['California', 'Texas', 'Illinois']

#### 3. The county  with the largest absolute change in population within the period 2010-2015

In [113]:
census_df = census_df.loc[:, ['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]
census_df['Absolute Change'] = census_df.apply(lambda row: max([row['POPESTIMATE2010'], row['POPESTIMATE2011'], row['POPESTIMATE2012'], row['POPESTIMATE2013'], row['POPESTIMATE2014'], row['POPESTIMATE2015']])
 - min([row['POPESTIMATE2010'], row['POPESTIMATE2011'], row['POPESTIMATE2012'], row['POPESTIMATE2013'], row['POPESTIMATE2014'], row['POPESTIMATE2015']]), axis=1)
census_df = census_df.set_index(['CTYNAME'])
census_df.sort_values(by='Absolute Change', ascending=False).index[0]

'Harris County'

#### 4.  Query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

In [115]:
c = census_df.loc[:, ['STNAME','CTYNAME', 'REGION','POPESTIMATE2014', 'POPESTIMATE2015']]
c['Comparison'] = c['POPESTIMATE2015'] > c['POPESTIMATE2014']
c = c[(c['REGION']==1) | (c['REGION']==2)]
c = c[c['CTYNAME'].str.startswith('Washington')]
c = c[c['Comparison']==True]
c = c.loc[:, ['STNAME', 'CTYNAME']]
c

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
